<a href="https://colab.research.google.com/github/Nov05/wandb-edu/blob/main/mlops-001/lesson1/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Info
notebook modified by nov05 on 2023-02-23  
go to [the github file](https://github.com/Nov05/wandb-edu/blob/main/mlops-001/lesson1/01_EDA_01.ipynb)  
view it at [nbviewer](https://nbviewer.org/github/Nov05/wandb-edu/blob/main/mlops-001/lesson1/01_EDA_01.ipynb)

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
!pwd

/content


In [ ]:
!wget https://raw.githubusercontent.com/Nov05/wandb-edu/2393150ad396928ab1421b0e2d6b259863348dcf/mlops-001/lesson1/utils.py
!wget https://raw.githubusercontent.com/Nov05/wandb-edu/2393150ad396928ab1421b0e2d6b259863348dcf/mlops-001/lesson1/params.py

In [ ]:
## do not install this. it is imported from local file "params.py".
## https://pypi.org/project/py-params/
# !pip install py-params
## Successfully installed py-params-0.10.2

In [ ]:
## https://pypi.org/project/wandb/
!pip install wandb
## Successfully installed wandb-0.13.10

<a href="https://colab.research.google.com/github/wandb/edu/blob/main/mlops-001/lesson1/01_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson1} -->

# EDA 
<!--- @wandbcode{course-lesson1} -->

In this notebook, we will download a sample of the [BDD100K](https://www.bdd100k.com/) semantic segmentation dataset and use W&B Artifacts and Tables to version and analyze our data. 

In [5]:
DEBUG = True # set this flag to True to use a small subset of data for testing

In [11]:
from fastai.vision.all import *
import params
import wandb

We have defined some global configuration parameters in the `params.py` file. `ENTITY` should correspond to your W&B Team name if you work in a team, replace it with `None` if you work individually. 

In the section below, we will use `untar_data` function from `fastai` to download and unzip our datasets. 

In [12]:
URL = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'

In [13]:
path = Path(untar_data(URL, force_download=True))

In [14]:
path.ls()
## [Path('/root/.fastai/data/bdd_simple_1k/LICENSE.txt'),Path('/root/.fastai/data/bdd_simple_1k/images'),
## Path('/root/.fastai/data/bdd_simple_1k/labels')]

(#3) [Path('/root/.fastai/data/bdd_simple_1k/LICENSE.txt'),Path('/root/.fastai/data/bdd_simple_1k/images'),Path('/root/.fastai/data/bdd_simple_1k/labels')]

Here we define several functions to help us process the data and upload it as a `Table` to W&B. 

In [15]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

We will start a new W&B `run` and put everything into a raw Artifact.

In [16]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="upload")
raw_data_at = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")
## If run into AttributeError: module 'params' has no attribute 'WANDB_PROJECT', 
## copy the file "param.py" to the working directory.

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [17]:
raw_data_at.add_file(path/'LICENSE.txt', name='LICENSE.txt')

ArtifactManifestEntry(path='LICENSE.txt', digest='X+6ZFkDOlnKesJCNt20yRg==', ref=None, birth_artifact_id=None, size=1594, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmppc1i45ee')

Let's add the images and label masks.

In [18]:
raw_data_at.add_dir(path/'images', name='images')
raw_data_at.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (/root/.fastai/data/bdd_simple_1k/images)... Done. 1.0s
wandb: Adding directory to artifact (/root/.fastai/data/bdd_simple_1k/labels)... Done. 0.5s


Let's get the file names of images in our dataset and use the function we defined above to create a W&B Table. 

In [19]:
image_files = get_image_files(path/"images", recurse=False)

# sample a subset if DEBUG
if DEBUG: image_files = image_files[:10]

In [20]:
table = _create_table(image_files, params.BDD_CLASSES)

Finally, we will add the Table to our Artifact, log it to W&B and finish our `run`. 

In [21]:
raw_data_at.add(table, "eda_table")

ArtifactManifestEntry(path='eda_table.table.json', digest='UGadhOjyDwnVIxyybUI7KA==', ref=None, birth_artifact_id=None, size=7692, extra={}, local_path='/root/.local/share/wandb/artifacts/staging/tmp0x_j6voy')

In [22]:
run.log_artifact(raw_data_at)
run.finish()